In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
prices = []
beds = []
baths = []
sizes = []
addresses = []

In [3]:
def scraping(page_numbers):
    pg_num = str(page_numbers)
    driver = webdriver.Chrome()
    url = "https://www.realtor.com/realestateandhomes-search/New-York_NY/pg-" + pg_num
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')
    for element in soup.findAll('li', attrs={'class': 'component_property-card'}):
       price = element.find('span', attrs={'data-label': 'pc-price'})
       bed = element.find('li', attrs={'data-label': 'pc-meta-beds'})
       bath = element.find('li', attrs={'data-label': 'pc-meta-baths'})
       size = element.find('li', attrs={'data-label': 'pc-meta-sqft'})
       address = element.find('div', attrs={'data-label': 'pc-address'})

       if bed and bath:
           nr_beds = bed.find('span', attrs={'data-label': 'meta-value'})
           nr_baths = bath.find('span', attrs={'data-label': 'meta-value'})

           if nr_beds and float(nr_beds.text) >= 2 and nr_baths :
               beds.append(nr_beds.text)
               baths.append(nr_baths.text)

               if price and price.text:
                   prices.append(price.text)
               else:
                   prices.append('No display data')

               if size and size.text:
                   sizes.append(size.text)
               else:
                   sizes.append('No display data')

               if address and address.text:
                   addresses.append(address.text)
               else:
                   addresses.append('No display data')

In [4]:
page_numbers = 15
for i in range(1,page_numbers,1):
    scraping(i)
df = pd.DataFrame({'Address': addresses, 'Price': prices, 'Beds': beds, 'Baths': baths, 'Sizes': sizes})
df.to_csv('scraped_data.csv', index=False, encoding='utf-8')